<a href="https://colab.research.google.com/github/COVID19Tracking/covid19-datafetcher/blob/master/tools/%5BCTP%5D_Long_Formulas_and_Cats_(CTP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CTP Publish Shift: Long-long-but-easy Formulas

### How to use:
Running a specific cell:  
click the ▶️ (play) button text to the cell, or `ctrl+Enter` the cell

Running the entire notebook:  
"Runtime" menu -> "Run all", or `ctrl-F9`


In [4]:
# DE positives
import pandas as pd
from datetime import datetime 
print("Last Run", datetime.now())
df = pd.read_csv('https://myhealthycommunity.dhss.delaware.gov/locations/state/download_covid_19_data')
df = df[df['Unit'] == 'tests'].set_index(['Year', 'Month', 'Day']).sort_index()
df.loc[df.index.unique()[-3]][['Statistic', 'Value']]


Last Run 2020-11-17 20:26:44.198226


Statistic     Value
Year Month Day                          
2020 11    14   Positive Tests   30580.0
           14      Total Tests  634013.0

In [ ]:
# HI PCR Test Encounters
import pandas as pd
from datetime import datetime
print("Last run at: ", datetime.now())
hi = pd.read_csv("https://public.tableau.com/views/EpiCurveApr4/CSVDownload.csv?:showVizHome=no")
hi.select_dtypes(exclude=['object']).sum()

Last run at:  2020-11-03 22:05:38.137102


New Cases                 15146
New Positive Tests        15167
Total Test Encounters    537421
dtype: int64

In [ ]:
# ID test


In [ ]:
# MA
from io import StringIO, BytesIO
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests
import zipfile

url = 'https://www.mass.gov/info-details/covid-19-response-reporting'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
a = soup.find('a', string=re.compile("COVID-19 Raw Data"))
link = "https://www.mass.gov{}".format(a['href'])
print("Download link = ", link)
res = requests.get(link)
zipdata = BytesIO(res.content)
zip = zipfile.ZipFile(zipdata, 'r')

df = pd.read_csv(zip.open('Testing2.csv'))[['Molecular Total']].iloc[-1]
print("Molecular Total People", df.sum())

df = pd.read_excel(BytesIO(zip.open('TestingByDate.xlsx', 'r').read())).filter(like="All Positive")
print(df.sum())

# weekly report
url = 'https://www.mass.gov/info-details/covid-19-response-reporting'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
a = soup.find('a', string=re.compile("Weekly Public Health Report - Raw"))
link = "https://www.mass.gov{}".format(a['href'])
print("\nWeekly link = ", link)
res = requests.get(link)
df = pd.read_excel(BytesIO(res.content), sheet_name='Antibody', parse_dates=['Test Date'], index_col='Test Date')
print(df.sum())

# ever hospitalized
print('\nEver Hospitalized')
foo = pd.read_excel(BytesIO(res.content), sheet_name='RaceEthnicity')
maxdate = foo['Date'].max()
foo[foo['Date'] == maxdate].sum()


Download link =  https://www.mass.gov/doc/covid-19-raw-data-november-15-2020/download
Molecular Total People 3037193
All Positive Molecular Tests                         232360.0
All Positive Molecular Tests_Higher Ed ONLY            3044.0
All Positive Molecular Tests_MA without Higher ED     77890.0
dtype: float64

Weekly link =  https://www.mass.gov/doc/weekly-public-health-report-raw-data-november-12-2020/download
Positive Tests                            11910.0
Total Tests                              129352.0
Unnamed: 3                                    0.0
DATA CURRENT AS OF 8:00AM ON 11/11/20         0.0
dtype: float64

Ever Hospitalized


Race/Ethnicity        Non-Hispanic WhiteNon-Hispanic Black/African A...
All Cases                                                        177541
Ever Hospitaltized                                                13588
Deaths                                                            10222
dtype: object

In [ ]:
# MI Testing
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173---,00.html'

req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
a = soup.find('a', string="Diagnostic Tests by Result and County")
mi_link = "https://www.michigan.gov/{}".format(a['href'])
print("Link = ", mi_link)

mi = pd.read_excel(mi_link).drop(columns=['COUNTY'])
mi.sum()

Link =  https://www.michigan.gov//documents/coronavirus/Diagnostic_Tests_by_Result_and_County_2020-11-03_706793_7.xlsx


Negative    4842727
Positive     249953
Total       5092680
dtype: int64

In [ ]:
# NC Antigen tests
import pandas as pd
nc = pd.read_csv("https://public.tableau.com/views/NCDHHS_COVID-19_DataDownload/DailyTestingMetrics.csv", parse_dates=['Date'], index_col='Date', thousands=',')
nc.pivot(columns='Measure Names').sum().astype('int64')

                Measure Names           
Measure Values  Antigen Tests                 23444
                Daily Tests Total           3890764
                Molecular (PCR) Tests       1036326
                Positive Test Percentage         15
dtype: int64

In [ ]:
# ND Negatives
import pandas as pd
import requests
from io import StringIO

url = "https://static.dwcdn.net/data/NVwou.csv"
headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0"}
req = requests.get(url, headers=headers)
data = StringIO(req.text)

nd = pd.read_csv(data)
nd.sum()

County                       AdamsBarnesBensonBillingsBottineauBowmanBurkeB...
Total Positive                                                           64885
Total Negative                                                          260606
Unique Individuals Tested                                               325491
Total Recovered                                                          53242
Deaths                                                                     743
dtype: object

In [ ]:
# ND other stuff:
import pandas as pd
nd = pd.read_csv("https://www.health.nd.gov/sites/www/files/documents/Files/MSS/coronavirus/charts-data/CovidTracking.csv")
nd.sum()['Total PCR tests (susceptible test encounters)']

726186

In [ ]:
# TX
import pandas as pd
url = 'https://www.dshs.texas.gov/coronavirus/TexasCOVID-19HospitalizationsOverTimebyTSA.xlsx'
df = pd.read_excel(url, sheet_name='COVID-19 ICU', skiprows=2)
print("ICU")
df.loc[df[df.columns[0]] == 'Total'][df.columns[-1]]

ICU


22    1879.0
Name: 2020-11-11, dtype: float64

In [ ]:
# UT 
from io import StringIO, BytesIO
import pandas as pd
import requests
import zipfile

url = 'https://coronavirus-dashboard.utah.gov/Utah_COVID19_data.zip'
res = requests.get(url)
zipdata = BytesIO(res.content)
zip = zipfile.ZipFile(zipdata, 'r')
for zf in zip.filelist:
  if zf.filename.startswith('Overview_Total Tests by Date'):
    # yay, the testing file
    title = 'Tests'
  elif zf.filename.startswith('Overview_Number of People Tested by Date'):
    title = 'People'
  else:
    title = None
  if title:
    title = "Metrics for {} (from {})".format(title, zf.filename)
    print(title, "\n"+"="*len(title))
    df = pd.read_csv(zip.open(zf.filename)).drop(columns=[' Total Daily Tests', 'Total Positive Tests', 'Daily People Tested', 'Daily Positive Tests'], errors="ignore")
    print(df.groupby(['Test Type', 'Result']).sum())

Metrics for People (from Overview_Number of People Tested by Date_2020-11-16.csv) 
                              Count
Test Type         Result           
Antigen by DFA/IF NEGATIVE    55241
                  POSITIVE    10527
PCR/amplification NEGATIVE  1042778
                  POSITIVE   145712
Metrics for Tests (from Overview_Total Tests by Date_2020-11-16.csv) 
                              Count
Test Type         Result           
Antigen by DFA/IF NEGATIVE   128260
                  POSITIVE    10890
PCR/amplification NEGATIVE  1458004
                  POSITIVE   157309


In [ ]:
# WI PCR Testing Encounters
import pandas as pd
from datetime import datetime
print("Last run at: ", datetime.now().isoformat())
wi = pd.read_csv("https://bi.wisconsin.gov/t/DHS/views/PercentPositivebyTestPersonandaComparisonandTestCapacity/TestCapacityDashboard.csv", thousands=",")
wi[wi['Measure Names'] == 'Total people tested daily']['Totals'].sum()

Last run at:  2020-10-21T21:48:10.156533


2924206